# Analyzing the Metadata of Publications in the Open Research Knowledge Graph 
This Jupyter notebook contains different analyses on the metadata of publications stored in the Open Research Knowledge Graph [ORKG](https://www.orkg.org/orkg/).

## Loading Data from the ORKG SPAQRL endpoint

In [18]:
%matplotlib widget
from SPARQLWrapper import SPARQLWrapper, CSV
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

ENDPOINT_URL = "https://www.orkg.org/orkg/triplestore"

PREFIXES =  """
            PREFIX orkgr: <http://orkg.org/orkg/resource/>
            PREFIX orkgc: <http://orkg.org/orkg/class/>
            PREFIX orkgp: <http://orkg.org/orkg/predicate/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            """

query = """
        SELECT ?paper, ?research_field, ?doi, ?title, ?author, ?orcid, ?month, ?year, ?url, ?venue
WHERE {
  ?paper a orkgc:Paper.
  OPTIONAL{?paper rdfs:label ?title.}
  OPTIONAL{?paper orkgp:P26 ?doi.}
  OPTIONAL{?paper orkgp:P30 ?field.
           ?field rdfs:label ?research_field.}
  OPTIONAL{?paper orkgp:P27 ?author_resrc.
           BIND(IF(isLiteral(?author_resrc), ?author_resrc, "") AS ?name1)
           OPTIONAL{?author_resrc rdfs:label ?author_label;
                            orkgp:HAS_ORCID ?orcid.}
           BIND(IF(BOUND(?author_label),?author_label, "") AS ?name2)
           BIND(IF(?name1 = "", ?name2, ?name1) AS ?author)
          }
  
  OPTIONAL{?paper orkgp:P28 ?month_resrc.
           BIND(IF(isLiteral(?month_resrc), ?month_resrc, "") AS ?month1)
           OPTIONAL{?month_resrc rdfs:label ?month_label.}
           BIND(IF(BOUND(?month_label),?month_label, "") AS ?month2)
           BIND(IF(?month1 = "", ?month2, ?month1) AS ?month)
          }
  
  OPTIONAL{?paper orkgp:P29 ?year_resrc.
           BIND(IF(isLiteral(?year_resrc), ?year_resrc, "") AS ?year1)
           OPTIONAL{?year_resrc rdfs:label ?year_label.}
           BIND(IF(BOUND(?year_label),?year_label, "") AS ?year2)
           BIND(IF(?year1 = "", ?year2, ?year1) AS ?year)
          }
  
  OPTIONAL{?paper orkgp:url ?url.}
  OPTIONAL{?paper orkgp:HAS_VENUE ?venue_resrc.
           ?venue_resrc rdfs:label ?venue.}
} ORDER BY ?paper
        """

user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])

sparql = SPARQLWrapper(ENDPOINT_URL, agent=user_agent)
sparql.setQuery(PREFIXES+query)
sparql.setReturnFormat(CSV)

try:
        results = sparql.queryAndConvert()
except Exception as e:
        print(e)

now = datetime.now()
with open('query_result_' + now.strftime('%Y-%m-%d') + '.csv', 'wb') as file:
        file.write(results)

## Initial Data Validation and Check
1. Reading the data and checking the shape and column names.

In [19]:
df = pd.read_csv('query_result_' + now.strftime('%Y-%m-%d') + '.csv', encoding='utf-8', encoding_errors='ignore')

print('Shape of the dataframe: '+ str(df.shape))
print('Column names: ' + str(df.columns))

Shape of the dataframe: (42216, 10)
Column names: Index(['paper', 'research_field', 'doi', 'title', 'author', 'orcid', 'month',
       'year', 'url', 'venue'],
      dtype='object')


2. Checking for NaN values in each column for the entire dataframe.

In [20]:
pd.set_option("max_rows", None)
for column in df:
    print(df[column].isna().value_counts(dropna=False).to_frame())

       paper
False  42216
       research_field
False           41998
True              218
         doi
False  35670
True    6546
       title
False  42206
True      10
       author
False   41806
True      410
       orcid
True   39145
False   3071
       month
False  32048
True   10168
        year
False  40420
True    1796
         url
True   33571
False   8645
       venue
False  26735
True   15481


3. Derterming the number of NaN entries of the column **paper** and the number of unique papers.

In [21]:
#print(df.drop_duplicates(subset='paper')['paper'].value_counts(dropna=False).sum())
print('Number of NaN entries: ' + str(df['paper'].isna().sum()))
print('Number of unique papers: '+ str(df['paper'].nunique()))

Number of NaN entries: 0
Number of unique papers: 10099


4. Checking the number of unique papers per **research field** and the number of unique papers without a research field.

In [22]:
df_unique_papers = df.drop_duplicates(subset='paper')
print('Number of unique papers without a research field: ' + str(df_unique_papers['research_field'].isna().sum()))
df_unique_papers['research_field'].value_counts().to_frame().head(15)

Number of unique papers without a research field: 118


,research_field
Science,3001
Bioinformatics,1191
Ecology and Evolutionary Biology,968
Information Science,550
Artificial Intelligence,321
Toxicology,295
Computer Sciences,243
Urban Studies and Planning,203
Medicinal Chemistry and Pharmaceutics,187
Natural Language Processing,164


5. Checking the number of incorrect entries for the column **doi** and the number of unique papers without a DOI.

In [23]:
incorrect_DOIs = df_unique_papers[~df_unique_papers['doi'].str.startswith('10', na=False)]
print('Number of incorrect DOIs :' + str(incorrect_DOIs['doi'].value_counts().sum()))
print(incorrect_DOIs['doi'].value_counts().to_frame().head(15))
print('Number of unique papers without a DOI: ' + str(df_unique_papers['doi'].isna().sum()))

Number of incorrect DOIs :50
                                                    doi
https://doi.org/10.1016/j.eswa.2019.05.052            1
 10.34865/mb311549d5_1                                1
https://doi.org/10.14778/2536336.2536343              1
hdl.handle.net/10520/AJA0038223X_3168                 1
https://doi.org/10.1016/j.procs.2016.09.123           1
doi.org/10.1016/j.artint.2012.03.006                  1
doi.org/10.1016/j.jbi.2013.09.008                     1
 10.34865/mb7934d5_2ad                                1
 10.34865/mb11207d5_1                                 1
 10.34865/mb10766kskd5_1                              1
 10.34865/mb7943verd5_1                               1
 10.34865/mb10001d5_1                                 1
 10.34865/mb0228fstd5_2ad                             1
https://www.nature.com/articles/s41598-019-40043-5    1
 10.34865/mb6772d5_2ad                                1
Number of unique papers without a DOI: 2256


6. Checking the number of unique papers with a specific **title** and the number of unique papers without a title.

In [24]:
#duplicated_title = df.drop_duplicates(subset=['paper'])
print('Number of unique papers without a title: ' + str(df_unique_papers['title'].isna().sum()))
df_unique_papers['title'].value_counts().loc[lambda x : x >= 2].to_frame().head(15)

Number of unique papers without a title: 8


,title
Governing nonprofit platform ecosystems – an information platform for refugees,14
Linking sea level rise and socioeconomic indicators under the Shared Socioeconomic Pathways,14
Absolute measurement of the resonance lines in heliumlike vanadium on an electron-beam ion trap,11
Creating the European Literary Text Collection (ELTeC): Challenges and Perspectives,9
Theoretical energies for the <i>n</i> = 1 and 2 states of the helium isoelectronic sequence up to <i>Z</i> = 100,6
International Phyical Performance Test Profile (IPPTP),5
Solution-processed high-performance p-channel copper tin sulfide thin-film transistors,5
Two New Phytoecdysteroids From Sphenocentrum jollyanum Pierre Root,4
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding,4
Model and heuristics for the Assembly Line Worker Integration and Balancing Problem,4


In [27]:
print('Number of unique papers without a month: ' + str(df_unique_papers['month'].isna().sum()))
print(df_unique_papers['month'].value_counts())

Number of unique papers without a month: 3116
1               1641
10               564
7                539
9                514
5                503
12               486
8                484
6                482
4                463
3                463
11               437
2                405
October 2020       1
0                  1
Name: month, dtype: int64


In [ ]:
print(md_df['year_number'].value_counts(dropna=False))

In [ ]:
field_df = md_df[['paper','field_label']].drop_duplicates()

pd.set_option("max_rows", None)
field_df[['field_label']].value_counts(dropna=False)

sns.countplot(y='field_label', data=field_df)

In [ ]:
df = pd.DataFrame()
nsfg['nbrnaliv'].replace([98,99], np.nan, inplace=True)

df['paper'] = md_df['paper'].astype('str')
df['field'] = md_df['field_label'].astype('category')
df['DOI'] = md_df['DOI'].astype('str')
df['title'] = md_df['title'].astype('str')
df['author'] = md_df['name'].astype('str')
df['orcid'] = md_df['id'].astype('str')
df['month'] = md_df['month_number'].fillna(0.0).astype('int')
df['year'] = md_df['year_number'].fillna(0.0).astype('int')
df['url'] = md_df['paper_url'].astype('str')
df['venue'] = md_df['venue_label'].astype('str')